## Hyperelasticity 

![hyperelasticity.png](figures/hyperelasticity.png)

In [2]:
using JuAFEM
using Tensors
using KrylovMethods
using TimerOutputs
import ProgressMeter
using ForwardDiff
using DiffResults
const ∇ = Tensors.gradient;

### NeoHook Material

In [3]:
immutable NeoHook{T}
    μ::T
    λ::T
end

# Potential, only needed when solving with AD
function Ψ(mp::NeoHook, E)
    C = 2E + one(E)
    Ic = trace(C)
    J = sqrt(det(C))
    lnJ = log(J)
    return mp.μ/2 * (Ic - 3) - mp.μ * lnJ + mp.λ / 2 * lnJ^2
end

# Second Piola Kirchoff
function compute_2nd_PK(mp::NeoHook, E)
    I = one(E)
    C = 2E + one(E)
    invC = inv(C)
    J = sqrt(det(C))
    return mp.μ *(I - invC) + mp.λ * log(J) * invC
end

# Tensors.jl AD to compute the ∂S∂E
function constitutive_driver(mp::NeoHook, E)
    ∂S∂E, SPK = ∇(E -> compute_2nd_PK(mp, E), E, :all)
    return SPK, ∂S∂E
end;

## Assembler routines

In [4]:
# Loop over all cells 
function do_assemble!{dim}(K, f, grid::Grid{dim}, dh::DofHandler, cv, fv, mp, u)
    n = ndofs_per_cell(dh)
    Ke = zeros(n, n)
    fe = zeros(n)

    assembler = start_assemble(K, f)

    # loop over all cells in the grid
    @timeit "assemble" for cell in CellIterator(dh)
        # reset
        fill!(Ke, 0)
        fill!(fe, 0)

        global_dofs = celldofs(cell)
        ue = u[global_dofs] # element dofs
        @timeit "inner assemble" assemble_element!(Ke, fe, cell, cv, fv, mp, ue)

        assemble!(assembler, global_dofs, fe, Ke)
    end
end;

In [5]:
# Assembles the contribution from the cell to ke and fe
function assemble_element!(ke, fe, cell, cv, fv, mp, ue)
    b = Vec{3}((0.0, -0.5, 0.0))
    t = Vec{3}((0.1, 0.0, 0.0))
    ndofs = getnbasefunctions(cv)
    reinit!(cv, cell)
    fill!(ke, 0.0)
    fill!(fe, 0.0)
    δE = Vector{SymmetricTensor{2, 3, eltype(ue), 6}}(ndofs)

    for qp in 1:getnquadpoints(cv)
        ∇u = function_gradient(cv, qp, ue)
        dΩ = getdetJdV(cv, qp)

        # strain and stress + tangent
        F = one(∇u) + ∇u
        E = symmetric(1/2 * (F' ⋅ F - one(F)))

        S, ∂S∂E = constitutive_driver(mp, E)

        # Hoist computations of δE
        for i in 1:ndofs
            δFi = shape_gradient(cv, qp, i)
            δE[i] = symmetric(1/2*(δFi'⋅F + F'⋅δFi))
        end

        for i in 1:ndofs
            δFi = shape_gradient(cv, qp, i)
            δu = shape_value(cv, qp, i)
            fe[i] += (δE[i] ⊡ S) * dΩ
            fe[i] -= (δu ⋅ b) * dΩ
            δE∂S∂E = δE[i] ⊡ ∂S∂E
            S∇δu = S ⋅ δFi'
            for j in 1:ndofs
                δ∇uj = shape_gradient(cv, qp, j)
                ke[i, j] += (δE∂S∂E ⊡ δE[j] + S∇δu ⊡ δ∇uj' ) * dΩ
            end
        end
    end

    for face in 1:nfaces(cell)
        if onboundary(cell, face)
            reinit!(fv, cell, face)
            for q_point in 1:getnquadpoints(fv)
                dΓ = getdetJdV(fv, q_point)
                for i in 1:ndofs
                    δu = shape_value(fv, q_point, i)
                    fe[i] -= (δu ⋅ t) * dΓ
                end
            end
        end
    end
end;

### Assembler routines -- AD

In [6]:
function do_assemble_AD!{dim}(K, f, grid::Grid{dim}, dh::DofHandler, cv, fv, mp, u)
    n = ndofs_per_cell(dh)
    Ke = zeros(n, n)
    fe = zeros(n)

    assembler = start_assemble(K, f)

    hessian_result = DiffResults.HessianResult(fe)
    hessian_config = ForwardDiff.HessianConfig(nothing, hessian_result, fe, ForwardDiff.Chunk(6))
    
    # loop over all cells in the grid
    @timeit "assemble" for cell in CellIterator(dh)
        global_dofs = celldofs(cell)
        ue = u[global_dofs] # element dofs

        ForwardDiff.hessian!(
            hessian_result,
            ue -> assemble_element(cell, cv, fv, mp, ue),
            ue,
            hessian_config)
        
         Ke = DiffResults.hessian(hessian_result)
         fe = DiffResults.gradient(hessian_result)
         assemble!(assembler, global_dofs, Ke, fe)
    end
    return
end;

In [7]:
function assemble_element(cell, cv, fv, mp, ue::Vector{T}) where {T}
    b = Vec{3}((0.0, -0.5, 0.0))
    t = Vec{3}((0.1, 0.0, 0.0))
    ndofs = getnbasefunctions(cv)
    reinit!(cv, cell)
    ∫ΨdΩ = zero(T)
    ∫budΩ = zero(T)
    ∫tudΓ = zero(T)

    for qp in 1:getnquadpoints(cv)
        u  = function_value(cv, qp, ue)
        ∇u = function_gradient(cv, qp, ue)
        dΩ = getdetJdV(cv, qp)
        F = one(∇u) + ∇u
        E = symmetric(1/2 * (F' ⋅ F - one(F)))
        ∫ΨdΩ += Ψ(mp, E) * dΩ
        ∫budΩ += (b ⋅ u) * dΩ
    end
    for face in 1:nfaces(cell)
        if onboundary(cell, face)
            reinit!(fv, cell, face)
            for qp in 1:getnquadpoints(fv)
                dΓ = getdetJdV(fv, qp)
                u = function_value(fv, qp, ue)
                ∫tudΓ += (u ⋅ t) * dΓ
            end
        end
    end
    return ∫ΨdΩ - ∫budΩ - ∫tudΓ
end;

## Main solver routine

In [8]:
function solve(; AD = false)
    reset_timer!()

    const dim = 3

    # Generate a grid
    N = 10
    L = 1.0
    left = zero(Vec{dim})
    right = L * ones(Vec{dim})
    grid = generate_grid(Tetrahedron, ntuple(x->N, dim), left, right)

    # Material parameters
    E = 10.0
    ν = 0.3
    μ = E / (2(1 + ν))
    λ = (E * ν) / ((1 + ν) * (1 - 2ν))
    mp = NeoHook(μ, λ)

    # finite element base
    ip = Lagrange{dim, RefTetrahedron, 1}()
    qr = QuadratureRule{dim, RefTetrahedron}(1)
    qr_face = QuadratureRule{dim-1, RefTetrahedron}(1)
    cv = CellVectorValues(qr, ip)
    fv = FaceVectorValues(qr_face, ip)

    # DofHandler
    dh = DofHandler(grid)
    push!(dh, :u, dim) # Add a displacement field
    close!(dh)

    function rotation(X, t, θ = deg2rad(60.0))
        x, y, z = X
        return t * Vec{dim}(
            (0.0,
            L/2 - y + (y-L/2)*cos(θ) - (z-L/2)*sin(θ),
            L/2 - z + (y-L/2)*sin(θ) + (z-L/2)*cos(θ)
            ))
    end

    dbcs = ConstraintHandler(dh)
    # Add a homogenoush boundary condition on the "clamped" edge
    dbc = Dirichlet(:u, getfaceset(grid, "right"), (x,t) -> [0.0, 0.0, 0.0], collect(1:dim))
    add!(dbcs, dbc)
    dbc = Dirichlet(:u, getfaceset(grid, "left"), (x,t) -> rotation(x, t), collect(1:dim))
    add!(dbcs, dbc)
    close!(dbcs)
    t = 0.5
    update!(dbcs, t)

    println("Analysis with ", length(grid.cells), " elements")

    # pre-allocate
    _ndofs = ndofs(dh)
    un = zeros(_ndofs) # previous solution vector
    u  = zeros(_ndofs)
    Δu = zeros(_ndofs)

    apply!(un, dbcs)

    K = create_sparsity_pattern(dh)
    f = zeros(_ndofs)

    newton_itr = -1
    NEWTON_TOL = 1e-8
    prog = ProgressMeter.ProgressThresh(NEWTON_TOL, "Solving:")

    while true; newton_itr += 1
        u .= un .+ Δu
        if AD
            do_assemble_AD!(K, f, grid, dh, cv, fv, mp, u)
        else
            do_assemble!(K, f, grid, dh, cv, fv, mp, u)
        end
        normg = norm(f[JuAFEM.free_dofs(dbcs)])
        apply_zero!(K, f, dbcs)
        ProgressMeter.update!(prog, normg; showvalues = [(:iter, newton_itr)])

        if normg < NEWTON_TOL
            break
        end

        if newton_itr > 30
            error("Reached maximum Newton iterations, aborting")
            break
        end

        @timeit "linear solve" ΔΔu, flag, relres, iter, resvec = cg(K, f; maxIter = 1000, tol = min(1e-3, normg))
        @assert flag == 0

        apply_zero!(ΔΔu, dbcs)
        Δu .-= ΔΔu
    end

    # save the solution
    @timeit "export" begin
        vtkfile = vtk_grid("hyperelasticity", dh)
        vtk_point_data(vtkfile, dh, u)
        vtk_save(vtkfile)
    end

    print_timer(linechars = :ascii)
    return u
end

solve (generic function with 1 method)

In [12]:
u = solve();

Analysis with 5000 elements


Solving: (thresh = 1e-08, value = 0.0163924)


 ---------------------------------------------------------------------------
                                    Time                   Allocations      
                            ----------------------   -----------------------
      Tot / % measured:          351ms / 83.3%           84.3MiB / 38.5%    

 Section            ncalls     time   %tot     avg     alloc   %tot      avg
 ---------------------------------------------------------------------------
 assemble                6    170ms  58.1%  28.3ms   26.1MiB  80.4%  4.35MiB
   inner assemble    30.0k    108ms  36.9%  3.59μs   19.2MiB  59.2%        -
 linear solve            5   76.3ms  26.1%  15.3ms    683KiB  2.05%   137KiB
 export                  1   46.1ms  15.8%  46.1ms   5.71MiB  17.6%  5.71MiB
 ---------------------------------------------------------------------------

Solving: Time: 0:00:00 (6 iterations)
  iter:  5


In [14]:
# This has a quite long compilation time (~40 seconds)
u_AD = solve(AD=true);

Analysis with 5000 elements


Solving: (thresh = 1e-08, value = 0.773794)
Solving: (thresh = 1e-08, value = 0.202129)
Solving: (thresh = 1e-08, value = 0.0163924)
Solving: (thresh = 1e-08, value = 0.000384975)
Solving: (thresh = 1e-08, value = 5.1712e-07)


 -----------------------------------------------------------------------
                                Time                   Allocations      
                        ----------------------   -----------------------
    Tot / % measured:        2.28s / 97.6%            130MiB / 59.9%    

 Section        ncalls     time   %tot     avg     alloc   %tot      avg
 -----------------------------------------------------------------------
 assemble            6    2.12s  95.0%   353ms   71.4MiB  91.8%  11.9MiB
 linear solve        5   70.7ms  3.17%  14.1ms    683KiB  0.86%   137KiB
 export              1   41.2ms  1.85%  41.2ms   5.71MiB  7.34%  5.71MiB
 -----------------------------------------------------------------------

Solving: Time: 0:00:02 (6 iterations)
  iter:  5


In [15]:
Base.Test.@test u_AD ≈ u
Base.Test.@test norm(u) ≈ 4.870833706518008
println("Hyperelasticity passed!")

Hyperelasticity passed!
